In [1]:
import pandas as pd 

def reshape_health_equity_data(input_path, output_path):
    # Load the dataset & change ZIP Code column to string to preserve appropriate format 
    data = pd.read_csv('Health_Equity_Hospital.csv', dtype={'ZIP Code': 'string', 
                                                            'Facility ID': 'string'})

    # Standardize Facility Name column to proper casing and fix apostrophes 
    data['Facility Name'] = data['Facility Name'].str.title().str.replace("'S", "'s")

    # Remove unnecessary columns
    columns_to_remove = ['Address', 'City/Town', 'County/Parish', 'Telephone Number', 'Start Date', 'End Date']
    data = data.drop(columns=columns_to_remove, errors='ignore')

    # Handle 'Not Available' rows with Footnote = 5
    perfect_score_rows = data[(data['Footnote'] == 5)]
    perfect_facilities = perfect_score_rows['Facility ID'].unique()

    # Update domain scores and question responses for facilities with perfect scores (footnote =5)
    for facility_id in perfect_facilities:
        # Update domain scores to 1 
        domain_rows = (data[(data['Facility ID'] == facility_id) & 
                             (data['HCHE Measure ID'].str.contains('HCHE_D[0-9]_F_SCORE', na=False))])
        data.loc[domain_rows.index, 'Score'] = 1

        # Update total score to 5 
        total_score_rows = (data[(data['Facility ID'] == facility_id) & 
                                 (data['HCHE Measure ID'] == 'HCHE_F_SCORE')])
        data.loc[total_score_rows.index, 'Score'] = 5

        # Update question responses to 'Yes'
        question_rows = (data[(data['Facility ID'] == facility_id) & 
                              (data['HCHE Measure ID'].str.contains('_F_RESPONSE', na=False))])
        data.loc[question_rows.index, 'Attestation Result'] = 'Yes'

     # Filter rows with domain scores and pivot
    domains_data = data[data['HCHE Measure ID'].str.contains('HCHE_D[0-9]_F_SCORE', na=False)]
    questions_data = data[data['HCHE Measure ID'].str.contains('HCHE_D[0-9]_Q[0-9]_F_RESPONSE', na=False)]
    total_scores_data = data[data['HCHE Measure ID'] == 'HCHE_F_SCORE']

    # Pivot questions into a columnar format
    questions_pivoted = questions_data.pivot_table(
        index=['Facility ID', 'Facility Name', 'State', 'ZIP Code'],
        columns='HCHE Measure ID',
        values='Attestation Result',
        aggfunc='first'
    ).reset_index()

    # Pivot domains into a columnar format
    reshaped_data = domains_data[['Facility ID', 'Facility Name', 'State', 'ZIP Code', 'HCHE Measure ID', 'Score']].pivot(
        index=['Facility ID', 'Facility Name', 'State', 'ZIP Code'],
        columns='HCHE Measure ID',
        values='Score'
    ).reset_index()   

    # Add total score to the reshaped dataset
    reshaped_data = reshaped_data.merge(
        total_scores_data[['Facility ID', 'Facility Name', 'State', 'ZIP Code', 'Score']].rename(columns={'Score': 'Total Score'}),
        on=['Facility ID', 'Facility Name', 'State', 'ZIP Code'],
        how='left'
    )

    # Merge in question responses
    reshaped_data = reshaped_data.merge(
        questions_pivoted,
        on=['Facility ID', 'Facility Name', 'State', 'ZIP Code'],
        how='left'
    )


    # Rename domain and question columns 
    reshaped_data = reshaped_data.rename(columns={
        'HCHE_D1_F_SCORE': 'D1',
        'HCHE_D2_F_SCORE': 'D2',
        'HCHE_D3_F_SCORE': 'D3',
        'HCHE_D4_F_SCORE': 'D4',
        'HCHE_D5_F_SCORE': 'D5',
        'HCHE_D1_Q1_F_RESPONSE': 'D1 Q1',
        'HCHE_D1_Q2_F_RESPONSE': 'D1 Q2',
        'HCHE_D1_Q3_F_RESPONSE': 'D1 Q3',
        'HCHE_D1_Q4_F_RESPONSE': 'D1 Q4',
        'HCHE_D2_Q1_F_RESPONSE': 'D2 Q1',
        'HCHE_D2_Q2_F_RESPONSE': 'D2 Q2',
        'HCHE_D2_Q3_F_RESPONSE': 'D2 Q3',
        'HCHE_D3_Q1_F_RESPONSE': 'D3 Q1',
        'HCHE_D4_Q1_F_RESPONSE': 'D4 Q1',
        'HCHE_D5_Q1_F_RESPONSE': 'D5 Q1', 
        'HCHE_D5_Q2_F_RESPONSE': 'D5 Q2',
    })

    # Save the cleaned and reshaped data
    reshaped_data.to_csv(output_path, index=False)
    print(f"Reshaped data saved to {output_path}")
    
input_path = 'Health_Equity_Hospital.csv'
output_path = 'Health_Equity_Hospital_Clean.csv'

reshape_health_equity_data(input_path, output_path)

Reshaped data saved to Health_Equity_Hospital_Clean.csv
